In order to make Python aware of ZEN Scripting DLLs one must register those in order to make them available for the COM interface. This is done be running the <b>regScripting_Release.bat</b> as adminsitrator. A detailed description on how to to this can be found inside the application note: <b>Control ZEN form Python</b>.

To run this notebook the follwing files are required:

- <b>regScripting.bat</b>

Additionally the package <b>czimage</b> is required. It contains the following files:

- <b>bftools.py</b>
- <b>czitools.py</b>
- <b>misctools.py</b>

This notebook demonstrates the capabilities of the .COM interface
used to establish a connection between ZEN Blue and Python.
This connection allows to use ZEN Blue OAD Simple-API within a python script.

Special thanks to Christoph Gohlke for providing the binaries on:

<style>a:link {color:#0000ff;background-color:transparent;text-decoration:none;}</style>
<a href="http://www.lfd.uci.edu/~gohlke/pythonlibs/">http://www.lfd.uci.edu/~gohlke/pythonlibs/</a>

Requirements:

- PyWin32: <a href="http://www.lfd.uci.edu/~gohlke/pythonlibs/#pywin32">http://www.lfd.uci.edu/~gohlke/pythonlibs/#pywin32</a>
- Bioformats: <a href="http://www.lfd.uci.edu/~gohlke/pythonlibs/#bioformats">http://www.lfd.uci.edu/~gohlke/pythonlibs/#bioformats</a>
- Javabridge: <a href="http://www.lfd.uci.edu/~gohlke/pythonlibs/#javabridge">http://www.lfd.uci.edu/~gohlke/pythonlibs/#javabridge</a>
- vLFD: <a href="http://www.lfd.uci.edu/~gohlke/pythonlibs/#vlfd">http://www.lfd.uci.edu/~gohlke/pythonlibs/#vlfd</a>
- Mahotas: <a href="https://pypi.python.org/pypi/mahotas">https://pypi.python.org/pypi/mahotas</a>
- Numpy: <a href="https://pypi.python.org/pypi/numpy">https://pypi.python.org/pypi/numpy</a>
- matplotlib: <a href="https://pypi.python.org/pypi/matplotlib/1.4.3">https://pypi.python.org/pypi/matplotlib/1.4.3</a>

The first step is to do the required imports and setting up the correct path to the required extra tools.

In [ ]:
import sys
import win32com.client
import os

sys.path.append(r'C:\Users\M1SRH\Documents\Python_Projects\Python_ZEN_Connection')
sys.path.append(r'C:\Users\M1SRH\Documents\Python_Projects\BioFormatsRead')
import ReadAnalyzeImageData as rad

savefolder = 'C:\\Python_ZEN_Output\\'
%matplotlib inline

Here the interesting part starts. The ZEN objetct containing the Simple API is imported into Python via the COM interface.

In [ ]:
## Import the ZEN OAD Scripting into Python
Zen = win32com.client.GetActiveObject("Zeiss.Micro.Scripting.ZenWrapperLM")

Now we can run the experiment from Pythin inside ZEN. It assumes that one alread configured the experiment within ZEN correctly. As the final step the CZI image data are save to disk, where it can be read via the BioFormtas library.

In [ ]:
## Define the experiment to be executed
ZEN_Experiment = 'COM_Control_96_Wellplate.czexp'

## run the experiment in ZEN and save the data to the specified folder
exp = Zen.Acquisition.Experiments.GetByName(ZEN_Experiment)
img = Zen.Acquisition.Execute(exp)

## Show the image in ZEN
Zen.Application.Documents.Add(img)

## Use the correct save method - it is polymorphic ... :)
filename = savefolder + img.Name
img.Save_2(filename)

Here the CZI image is read into an 6D NumPy array. This example assumes XYCZT dimension order to be used during the acquisition. Of course this can be adapted to your needs.during the acquisition. The resulting NumPy array for XYCZT has the following shape:
    
<b>[series, time, zplane, channel, x, y]</b>

BioFormtas reads the CZI image <b>and</b> all important metainformation.

And finally al images are analzed using the Mahota image processing library and the result is plotted. The experiment acquired one single image per well. The cell number was counted the the result is displyed as a heatmap.

In [ ]:
## get the actual CZI image data using Python wrapper for BioFormats
img6d = rad.ReadImage(filename)

## Analyze the images - Example: Count Cells
obj, labeled = rad.CountObjects(img6d)

## Display some data
rad.DisplayData2(obj, labeled)